In [ ]:
!pip install xgboost

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

In [ ]:
folder_path = "/content/data"

In [ ]:
dataList = []

In [ ]:
for filename in os.listdir(folder_path):
  file_path = os.path.join(folder_path, filename)
  df = pd.read_excel(file_path)
  dataList.append(df)

In [ ]:
for i, df in enumerate(dataList):
  df.insert(len(df.columns), "pathogen", df.columns[0])

In [ ]:
for i in range(7):
  print(dataList[i].columns[0])

In [ ]:
dataList[0]

In [ ]:
dataList = [df.drop(df.columns[[0,1,2]], axis=1) for df in dataList]

In [ ]:
dataList[0]

In [ ]:
df_data = pd.concat(dataList, ignore_index=True)

In [ ]:
df_data

In [ ]:
df_data.shape

In [ ]:
df_data.isnull().sum()

In [ ]:
df_data.columns

In [ ]:
for column in df_data.columns:
    df_data[column].fillna(df_data[column].mode()[0], inplace=True)

In [ ]:
df_data

In [ ]:
df_data.isnull().sum()

In [ ]:
df_data.dtypes

In [ ]:
df_data=df_data.astype('str')

In [ ]:
X = df_data.iloc[:, :-1]
y = df_data.iloc[:, -1]

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
enc = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

X_train_encoded = enc.fit_transform(X_train)
X_test_encoded = enc.transform(X_test)

In [ ]:
feature_names = enc.get_feature_names_out(X_train.columns)

X_train = pd.DataFrame(X_train_encoded, columns=feature_names)
X_test = pd.DataFrame(X_test_encoded, columns=feature_names)

In [ ]:
y_train=y_train.replace({
    "vibrio": 0,
    "Yersinia Ruckeri": 1,
    "V.Salmoninarum": 2,
    "Aeromonas spp.": 3,
    "Flavobacter psychrophilum": 4,
    "L.garvieae": 5,
    "Pseudomonas spp.": 6,
})

y_test=y_test.replace({
    "vibrio": 0,
    "Yersinia Ruckeri": 1,
    "V.Salmoninarum": 2,
    "Aeromonas spp.": 3,
    "Flavobacter psychrophilum": 4,
    "L.garvieae": 5,
    "Pseudomonas spp.": 6,
})

In [ ]:
X_train.columns

In [ ]:
model = xgb.XGBClassifier(
    max_depth=3,
    n_estimators=50
    )

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)
predictions

In [ ]:
predictions.shape

In [ ]:
xgb.plot_importance(model, importance_type='weight')

In [ ]:
accuracy = accuracy_score(y_test, predictions) * 100
print("Accuracy: %.2f" % (accuracy))

In [ ]:
# Confusion matrix
sns.set(font_scale=1.4)
sns.heatmap(conf_mat, annot=True, cmap='Blues', fmt='d', cbar=True, xticklabels=categories, yticklabels=categories)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)

plt.show()